<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_CNN_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Data

In [1]:
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 106kB/s 
     |████████████████████████████████| 3.1MB 49.9MB/s 
     |████████████████████████████████| 501kB 43.3MB/s 


In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import seaborn as sns

In [0]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.models import Model

In [4]:
tf.__version__

'2.0.0-beta1'

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
M, I, J, Ch = X_train.shape

In [0]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [0]:
X_train, X_test = X_train / np.max(X_train), X_test / np.max(X_test) # scaling

In [0]:
K = len (set(y_train))

### Dimension Check

In [19]:
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000,) (10000,)


# Model Building

*Using functional API : better for split and multi-input / output*

In [0]:
i = Input (shape=(I,J,Ch))

In [0]:
c = 32
x = Conv2D (c, kernel_size=(3,3), strides=2, activation='relu')(i)
x = Conv2D (c*2, kernel_size=(3,3), strides=2, activation='relu')(x)
x = Conv2D (c*4, kernel_size=(3,3), strides=2, activation='relu')(x)
x = Flatten ()(x)
x = Dropout (0.5)(x)
x = Dense (1024, activation='relu')(x)
x = Dropout (0.5)(x)
x = Dense (K, activation='softmax')(x)

In [0]:
model = Model (i, x)

In [0]:
adam = tf.keras.optimizers.Adam (learning_rate=0.0001, decay=1e-6)

In [0]:
model.compile (optimizer=adam,
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
%%capture
r = model.fit (X_train, y_train, validation_data=(X_test, y_test), epochs=10)

In [0]:
print ('Training Set Evaluation : ' + str(model.evaluate (X_train, y_train)))
print ('Test Set Evaluation : ' + str(model.evaluate (X_test, y_test)))

In [0]:
plt.plot (r.history['loss'], label='loss', color='#840000')
plt.plot (r.history['val_loss'], label='validation loss', color='#00035b')
plt.legend ()

In [0]:
plt.plot (m.history['accuracy'], label='accuracy', color='#840000')
plt.plot (m.history['val_accuracy'], label='validation accuracy', color='#00035b')
plt.legend ()

In [0]:
X_test_100 = X_test[:100]
y_pred_100 = model.predict_classes (X_test_100)

### Misclassification Samples

In [0]:
misclass = np.where (y_pred_100 != y_test_100)[0]

In [0]:
i = np.random.choice (misclass)

In [0]:
plt.imshow (X_test[i], cmap='Blues')
plt.title ('True: %s, Predicted: %s' % (y_test[i], y_pred[i]))